In [21]:
####
#Clean Porn Dataset
#Tianhao
#2021/7/29
####

import pandas as pd
import datetime
import numpy as np
import xlsxwriter

In [28]:
### Read Categories File
pornhub_categories = pd.read_excel(r'C:\Users\raymo\Documents\GitHub\Callander-Research\Porn database\Data\RAW\PH_gay\Categories.xlsx')

### Read xlsx File
local_path = 'C:/Users/raymo/Documents/GitHub/Callander-Research/Porn database/Data/RAW/PH_gay/' #local path of where the raw data stored
store_path = 'C:/Users/raymo/Documents/GitHub/Callander-Research/Porn database/Data/OUTPUT/' #local path of where the output data stored
file_names = ['2021-03-07.xlsx','2021-03-14.xlsx','2021-03-21.xlsx','2021-03-28.xlsx','2021-04-04.xlsx','2021-04-18.xlsx'
             ,'2021-05-16.xlsx','2021-05-30.xlsx','2021-06-13.xlsx','2021-06-20.xlsx','2021-07-18.xlsx','2021-07-25.xlsx'
             ] #file name

datasets = {}
for file in file_names:
    sample = pd.read_excel(local_path+file,engine=None) #read in files
    datasets[file] = sample



C:\Users\raymo\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [29]:
sample = pd.read_excel('C:/Users/raymo/Documents/GitHub/Callander-Research/Porn database/Data/RAW/PH_gay/2021-06-20.xlsx',engine=None)

In [78]:
#Series types
#This function convert series into correct types and check any null

def convert_types(file_name, data):
    copy_data = data.copy()

    #extract the viewkeys from pornhub urls
    
    copy_data["url"] = [i.replace("https://www.pornhub.com/view_video.php?viewkey=","") for i in copy_data["url"]]
    
    #convert views(str) to int
    copy_data["views"] = copy_data["views"].fillna('0')
    copy_data["views"] = [int(str(i).replace(',' , '')) for i in copy_data["views"]]
    
    #convert approval to int
    copy_data["approval"] = copy_data["approval"].fillna('0')
    copy_data["approval"] = [int(str(i).replace('%',''))/100 for i in copy_data["approval"]]
    
    
    #convert upload_date to date_time
    extract_time_str = file_name.replace(".xlsx","").split("-")
    extract_time = datetime.date(int(extract_time_str[0]),int(extract_time_str[1]),int(extract_time_str[2]))
    upload_time = np.array([])
    for i in copy_data["upload_date"]:
        copy_data["upload_date"] = copy_data["upload_date"].fillna('0 years ago')
        temp = i.split(" ")
        if temp[0] == 'Yesterday':
            upload_time = np.append(upload_time,extract_time- datetime.timedelta(days= 1))
        elif temp[1] == 'week' or temp[1] == 'weeks':
            upload_time = np.append(upload_time,extract_time- datetime.timedelta(days= int(temp[0])*7))
        elif temp[1] == 'month' or temp[1] == 'months':
            upload_time = np.append(upload_time,extract_time- datetime.timedelta(days= int(temp[0])*30))
        elif temp[1] == 'year' or temp[1] == 'years':
            upload_time = np.append(upload_time,extract_time- datetime.timedelta(days= int(temp[0])*365))
        elif temp[1] == 'day' or temp[1] == 'days':
            upload_time = np.append(upload_time,extract_time- datetime.timedelta(days= int(temp[0])*1))
        else:
            upload_time = np.append(upload_time,'')
    copy_data["upload_date"] = upload_time
    
    #Convert all categories into string and check null
    copy_data["categories"] = [str(i) for i in copy_data["categories"]]
    
    #convert approval_pos and neg to int 
    copy_data["approval_pos"] = copy_data["approval_pos"].fillna('0')
    copy_data["approval_pos"] = [int(str(i).replace('%','').replace('K','000')) for i in copy_data["approval_pos"]]
    copy_data["approval_neg"] = copy_data["approval_neg"].fillna('0')
    copy_data["approval_neg"] = [int(str(i).replace('%','').replace(".0","").replace('K','000')) for i in copy_data["approval_neg"]]
    
    #Calculate True approval rate
    copy_data["approval"] = round(copy_data["approval_pos"]/(copy_data["approval_pos"]+copy_data["approval_neg"]),4)
    
    
    #Clean Actors Column
    copy_data["actors"] = copy_data["actors"].fillna('')
    copy_data["actors"] = [i.replace("Pornstars: ",'').replace("Suggest ",'').replace("\n ",'')
        .replace("Thank you for your suggestions! Our team is reviewing them!","") for i in copy_data["actors"]]
    copy_data["actors"] = ['NaN' if i == '' else i for i in copy_data["actors"]]

    #clean comment number
    copy_data["comments_number"] = copy_data["comments_number"].fillna('0')
    copy_data["comments_number"] = [int(i.replace('(','').replace(')','')) for i in copy_data["comments_number"]]
    
    #get bag of words for categories
    categories_data = modify_categories(copy_data)
    
    #Clean Comments
    comments = copy_data["comments_text"].fillna('')
    all_comments = []
    for c in comments:
        c_only = []
        temp = c.split(";")
        for i in temp:
            if '\n' not in i:
                c_only.append(i)
        all_comments.append(c_only)
    for i in np.arange(len(comments)):
        if comments[i] == [""]:
            comments[i] = []
    copy_data["comments_text"] = all_comments
    
    #create new table for comments
    comments_table = pd.DataFrame(columns = ['url','comments_number','comments_text'])
    comments_table['url'] = copy_data["url"]
    comments_table['comments_number'] = copy_data["comments_number"]
    comments_table['comments_text'] = copy_data["comments_text"]
    
    
    #clean related playlists videostill
    temp1 = []
    related_list = copy_data["related_videos_videostill"].fillna("")
    for i in related_list:
        temp1.append(i.split(';'))
    copy_data["related_videos_videostill"] = temp1
    
    #clean related playlists titles related_videos_videostill_alt
    temp2 = []
    related_list = copy_data["related_videos_videostill_alt"].fillna("")
    for i in related_list:
        temp2.append(i.split(';'))
    copy_data["related_videos_videostill_alt"] = temp2
    
    #create new table for related playlists
    related_videos = pd.DataFrame(columns = ['url','related_videos_videostill','related_videos_videostill_alt'])
    related_videos['url'] = copy_data['url']
    related_videos['related_videos_videostill'] = temp1
    related_videos['related_videos_videostill_alt'] = temp2
    
    #drop related_videos
    copy_data = copy_data.drop(columns = ["related_playlists_videostill_alt","related_playlists_videostill","related_videos_videostill","related_videos_videostill_alt",
                                          "related_videos","videostill_image_alt","categories","comments_text","comments_number"])
    
    return copy_data,categories_data,related_videos,comments_table


In [79]:
# This function create the bag of words for categories
def modify_categories(data):
    new_categories = [i.replace("Categories \n ","").replace(" \n Suggest","") for i in data["categories"]]
    bag = np.array([[]])
    for fullstring in data["categories"]: 
        temp = np.array([])
        for substring in pornhub_categories["Categories"]:
                
            if substring in fullstring:
                temp = np.append(temp, 1)
            elif substring not in fullstring:
                temp = np.append(temp, 0)
        
        bag = np.append(bag,temp)  
    bag = bag.reshape(data["categories"].size,pornhub_categories["Categories"].size)
    bag = pd.DataFrame(bag,index = data["url"] ,columns = pornhub_categories["Categories"])
    return bag



In [80]:
convert_types("2021-2-2",sample)[0]

,url,title,views,approval,upload_date,approval_pos,approval_neg,uploaded_user,actors,production,tags,related_playlists,videostill_image,videostill_image_filename,_url_input
0,ph606c9b210339e,Fucked Hard Doggy Style at the Window of a Cap...,23033,0.9512,2020-12-04,156,8,Falcon Al,NaN,homemade,Tags \n big cock анал gay гей домашка Жесткий ...,XL BB sessions (part 1); AMATEUR; Uncut 3; bar...,https://di.phncdn.com/videos/202104/06/3862300...,videostill_image/(m=eaf8Ggaaaa)(mh=5NqiJJ9XbWs...,https://www.pornhub.com/view_video.php?viewkey...
1,ph5db00da91be01,Fatty Jerking off on the Couch,2659,0.9600,2020-02-03,24,1,Daddy Nox,NaN,homemade,Tags \n masturbate old gay jerking off small d...,NaN,https://ei.phncdn.com/videos/201910/23/2566842...,videostill_image/(m=eaf8Ggaaaa)(mh=dZSoJ7TP-29...,https://www.pornhub.com/view_video.php?viewkey...
2,ph60ace024ad1a7,SOUTHERNSTROKES Christian Hermes and Nick Star...,5159,0.8065,2021-01-12,25,6,maniamen,NaN,professional,Tags \n southernstrokes gay young gay twink bl...,NaN,https://ei.phncdn.com/videos/202105/25/3885710...,videostill_image/(m=eaf8Ggaaaa)(mh=XchgUtHxm99...,https://www.pornhub.com/view_video.php?viewkey...
3,ph5f7f6e3e7df61,Hammock Blowjob with Cum Shot,6917,0.8298,2020-06-07,39,8,hunguncutfun,NaN,homemade,Tags \n gay outdoor hung deep throat cock ring...,Uncut 3; Oral; SUPERSUCKER 5; cock suckers; ch...,https://di.phncdn.com/videos/202010/08/3590533...,videostill_image/(m=eaf8Ggaaaa)(mh=Lg5iJlASj-m...,https://www.pornhub.com/view_video.php?viewkey...
4,ph5f2ca9248b63e,Up Close with my Uncut Cock & Cum,3175,1.0000,2020-11-04,15,0,Sai Arya,NaN,homemade,Tags \n masturbate verified ama foreskin fores...,NaN,https://ei.phncdn.com/videos/202008/07/3401095...,videostill_image/(m=eaf8Ggaaaa)(mh=qlT20wKHznL...,https://www.pornhub.com/view_video.php?viewkey...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,ph6086d8a953051,Chavs Fuck at Crusin Spot - Onlyfans/northwest...,88433,0.8913,2021-01-03,443,54,NorthWestChavs,NaN,homemade,Tags \n chav rimming bareback gay scally twink...,Young Muscle; spandex; Raw seeding; Cruising; ...,https://ei.phncdn.com/videos/202104/26/3871599...,videostill_image/(m=eaf8Ggaaaa)(mh=Tb28T4g2TXJ...,https://www.pornhub.com/view_video.php?viewkey...
993,ph60153fccb8aae,Beautiful Femboy Fucks his own Ass and Cums Big,66316,0.8901,2020-10-05,243,30,prettyboi2000x,NaN,homemade,Tags \n gay femboy femboy fucked dildo anal di...,Young Muscle; spandex; ggg; gggg; dildo fuckin...,https://ei.phncdn.com/videos/202101/30/3826894...,videostill_image/(m=eaf8Ggaaaa)(mh=MghUQCPfcGF...,https://www.pornhub.com/view_video.php?viewkey...
994,ph605311083e8f8,Kinky Tattooed Amateur Masturbates his Thick M...,7201,0.8600,2020-11-04,43,7,Alternadudes,NaN,professional,Tags \n amateurarmpits pitsandpubes gay big co...,Young Muscle; spandex; Daddy; Hairy chest 3; H...,https://di.phncdn.com/videos/202103/18/3853068...,videostill_image/(m=eaf8Ggaaaa)(mh=eEZNiHpqrfM...,https://www.pornhub.com/view_video.php?viewkey...
995,ph5ee4d450bcbc2,Zoom Classmates Jerk off together | Athletes M...,148368,0.8720,2020-02-03,511,75,JockTheBlackThot,NaN,homemade,Tags \n black bbc gay amateur college raw gay ...,Young Muscle; spandex; College; cumpilation; H...,https://ei.phncdn.com/videos/202006/13/3233344...,videostill_image/(m=eaf8Ggaaaa)(mh=B-04WOpDXcz...,https://www.pornhub.com/view_video.php?viewkey...


In [33]:
sample["comments_text"]

0      rob33 \n \n \n 1 month ago; 69mussy \n \n \n 2...
1      Love4chubbs96 \n \n \n 1 year ago; SashaFemGuy...
2                                                    NaN
3                         ABenoit69 \n \n \n 1 month ago
4      fighterxmen \n \n \n 3 months ago; mgg1959 \n ...
                             ...                        
992    Yfrontslad \n \n \n 1 month ago; GayBoy212020 ...
993    Swaimtrain \n \n \n 3 weeks ago; Passionate_fu...
994    Swaimtrain \n \n \n 3 weeks ago; lonjapuff \n ...
995    medic715 \n \n \n 1 year ago; Penguin230 \n \n...
996                         Ukpmart \n \n \n 1 month ago
Name: comments_text, Length: 997, dtype: object

In [146]:
x = 0
np.isnan(x)

False

In [ ]:
#export modified dataset

for k in datasets.keys():
    modified = convert_types(k,datasets[k])
    
    #write out excel file
    modified[0].to_excel(store_path+"OUTPUT_" + k,sheet_name = 'Sheet 1')
    modified[1].to_excel(store_path+"OUTPUT_Categories_" + k,sheet_name = 'Sheet 1')
    modified[2].to_excel(store_path+"OUTPUT_Related_Videos_" + k,sheet_name = 'Sheet 1')
    modified[3].to_excel(store_path+"OUTPUT_Comments_" + k,sheet_name = 'Sheet 1')
    print("Output for file: ", k, " is successful :) ")

Output for file:  2021-03-07.xlsx  is successful :) 
Output for file:  2021-03-14.xlsx  is successful :) 
Output for file:  2021-03-21.xlsx  is successful :) 
